<!-- $\left( \theta + a \sin \omega t \right)^{2}$

$\theta^{2} + 2 \theta a \sin \omega t + a^{2} \sin^{2} \omega t $

$\theta^{2} + 2 \theta a \sin \omega t + + a^{2} \frac{1}{2} \left( 1 - \cos 2 \omega t \right)$ -->

# <center> Extremum Seeking </center>

Extremum Seeking (ES) is a model-free optimization and control technique, that can be used in real-time. ES operates to minimize of maximize an objective function value, which is either fed into the ES algorithm, or calculated from measurements fed into th ES algorithm. ES approximates gradient descent, in that it estimates the gradient of the objective function with respect to its setpoint (system input), and moves its setpoint accordingly.


## <center> ES Operation </center>

ES approximates gradient descent by steering its optimizer estimate, $\hat{\theta}$, with an estimate of the gradient of an objective function to $\hat{\theta}$.

ES operates as follows:

The system input is formed by adding a sinusoidal perturbation $a \sin \left( \omega t \right)$ to the ES setpoint $\hat{\theta}$, such that $\theta = \hat{\theta} + a \sin \left( \omega t \right)$. The setpoint is the estimate of the (local) optimizer.

The system input $\theta$ passes through the unknown system, resulting in system output (measurements) $\textbf{y} \left( u \right)$.

Either a central entity computes the objective function value $\Psi = \Psi \left( \bf{y} \right)$, or the ES algorithm may take in the measurements $\bf{y}$, and compute $\Psi$ itself.

A high-pass filter removes low-frequency content from $\Psi$, giving $\rho$

This value is demodulated by multiplying by $\sin \left( \omega t \right)$, giving $\sigma$.

A low-pass filter removes high-frequency content from this value, giving the gradient estimate, $\hat{\xi}$.

The gradient estimate is integrated back into the setpoint (optimizer estimate), updating $\hat{\theta}$.


## <center> Extremum Seeking Parameters </center>

This section describes the parameters of an instance of 1D ES with sinusoidal perturbation.

### Parameters for Simple ES

#### Perturbation Frequency: $f$ [Hz]
The perturbation frequency $f$ determines how fast the sinusoidal perturbation cycles. This is converted into an angular frequency $\omega = 2 \pi f$.

#### Perturbation Amplitude: $a$
The perturbation amplitude $a$ determines how far into the search space the perturbation reaches. Larger values tend to provide faster convergence to an optimizer, as the ES can assess more of the search space. Smaller values may provide slower convergence to an optimizer, but the control value will remain closer to the optimizer after convergence.

#### Integrator Gain: $b$
The integerator gain $b$ determines how far the ES algorithm will move in the direction of the gradient estimate. This value essentially scales the gradient estimate. This value can be though of as similar to the step size in gradient descent. Larger values generally provide faster convergence to an optimizer, but may lead to instability.

### Parameters for Advanced ES

#### High-pass filter frequency: $\omega_h$
The high-pass filter frequency determines what frequency content passes through the high-pass filter. This value is typically set to $\omega_{h} = 0.1 \omega$, and SimpleES classes default to this value.

#### Low-pass filter frequency: $\omega_l$
The low-pass filter frequency determines what frequency content passes through the low-pass filter. The low-pass filter attenuates content in the demodulated signal, and typically is used to attenuate content such as that due to the perturbation. This value is typically set to $\omega_{l} = 0.1 \omega$, and SimpleES classes default to this value.


## <center> Inside the ES algorithm </center>

Here, we define $s$ as the Laplace variable, $k$ as the index for discretized equations, and $T$ as the discretized timestep.

#### Objective function: $\Psi$
The ES algorithm received the objective function value $\Psi$

#### $\rho$: Changes in the objectuve function due to the perturbation
The objective function value, $\Psi$, passes through a high-pass filter to remove low frequency content, such as changes in the objective function value due to changes in the setpoint. Changes in the objective function value due to the sinusoidal perturbation pass through the filter. The output of the high-pass filter is $\rho$:

<center>
Continuous representation: $\rho = \displaystyle \frac{s}{s + \omega_{h}} \Psi$ $\quad$ | $\quad$ Discretized representation, indexed by $k$: $\rho_{k} = \left( 1 - T \omega_{h} \right) \rho_{k-1} + \Psi_{k} - \Psi_{k-1}$
$\quad$
</center>


#### $\epsilon$: Changes in the objective function due to the setpoint
A pertinent value is $\epsilon$, which is the difference between the objective function value and the output of the high-pass filter. This can be viewed as the objective function component due to the setpoint $\hat{\theta}$:

<center>
Continuous representation: $\epsilon = \Psi - \rho$ $\quad$ | $\quad$ Discretized representation, indexed by $k$: $\epsilon_{k} = \Psi_{k} - \rho_{k}$
<!-- <center> Discretized representation, indexed by $k$: $\epsilon_{k} = \Psi_{k} - \rho_{k}$ -->
$\quad$
</center>


#### $\sigma$: Demodulated value
The high-pass filtered objective function value $\rho$ then is demodulated by multiplying it by the sinusoidal perturbation, and dividing by the perturbation ampltiude, giving $\sigma$:

<center> Continuous representation: $\sigma = \displaystyle \frac{2}{a} \sin \left( \omega t \right) \rho$ $\quad$ | $\quad$ Discretized representation, indexed by $k$: $\sigma_{k} = \displaystyle \frac{2}{a} \sin \left( \omega k T \right) \rho_{k}$
<!-- <center> Discretized representation, indexed by $k$: $\sigma_{k} = \displaystyle \frac{2}{a} \sin \left( \omega k T \right) \rho_{k}$ -->
$\quad$
</center>


#### $\hat{\xi}$: Gradient Estimate
The demodulated value passes through a low-pass filter to remove sinsoidal and other high frequency content, giving an estimate of the gradient of the objective function $\Psi$ with respect to the setpoint $\hat{\theta}$, $\hat{\xi}$:

<center> Continuous representation: $\hat{\xi} = \displaystyle \frac{\omega_{l}}{s + \omega_{l}} \sigma$ $\quad$ | $\quad$ Discretized representation, indexed by $k$: $\hat{\xi}_{k} = \left( 1 - T \omega_{l} \right) \hat{\xi}_{k-1} + T \omega_{l} \sigma_{k-1}$
<!-- <center> Discretized representation, indexed by $k$: $\hat{\xi}_{k} = \left( 1 - T \omega_{l} \right) \hat{\xi}_{k-1} + T \omega_{l} \sigma_{k-1}$ -->
$\quad$
</center>

#### $\hat{\theta}$: Setpoint
The ES algotrithm then integrates its gradient estimate, scaled by a gain $b$, to update its setpoint $\hat{\theta}$. Positive values of $b$ are used to maximize $\Psi$, and negative values of $b$ are used to minimize $\Psi$ :

<center> Continuous representation: $\hat{\theta} = \displaystyle \pm \frac{b}{s} \hat{\xi}$ $\quad$ | $\quad$ Discretized representation, indexed by $k$: $\hat{\theta}_{k} = \hat{\theta}_{k-1} \pm \displaystyle b T \hat{\xi}_{k-1}$
<!-- <center> Discretized representation, indexed by $k$: $\hat{\theta}_{k} = \hat{\theta}_{k-1} \pm \displaystyle b T \hat{\xi}_{k-1}$ -->
$\quad$
</center>


#### $\theta$: Control
The ES algotrithm adds the perturbation to its setpointto update its setpoint $\hat{\theta}$, giving the control value, $\theta$:

<center> Continuous representation: $\theta = \hat{\theta} + a \sin \left( \omega t \right)$ $\quad$ | $\quad$ Discretized representation, indexed by $k$: $\theta_{k} = \hat{\theta}_{k} + a \sin \left( \omega k T \right)$
<!-- <center> Discretized representation, indexed by $k$: $\hat{\theta}_{k} = \hat{\theta}_{k-1} \pm \displaystyle b T \hat{\xi}_{k-1}$ -->
$\quad$
</center>



## <center> Extremum Seeking Gradient Estimation </center>

Here, we take advantage of second order Taylor Expansion: where $f(b) \approx f (a) + \displaystyle \left. \frac{df}{dx} \right|_{x = a} \left( b - a \right) + \left. \frac{1}{2} \frac{d^{2}f}{dx^{2}} \right|_{x = a} \left( b - a \right)^{2}$

Using a first order Taylor Expansion, the objective function can be expressed in two parts, a portion due to the setpoint, and a portion due to the perturbation:

<center>
$\Psi ( \theta ) \approx \Psi ( \hat{\theta} ) + \displaystyle \left. \frac{d \Psi}{d \theta} \right|_{\theta = \hat{\theta}} \left( \theta - \hat{\theta} \right) = \Psi ( \hat{\theta} ) + \displaystyle \left. \frac{d \Psi}{d \theta} \right|_{\theta = \hat{\theta}} a \sin \left( \omega t \right)$
$\quad$
</center>

The high-pass filter removes "slow" and low frequency content, such as the portion of the objective function due to the setpoint:

<center>
$\rho \approx \displaystyle \left. \frac{d \Psi}{d \theta} \right|_{\theta = \hat{\theta}} a \sin \left( \omega t \right)$
$\quad$
</center>

This signal is demodulated by multiplying by $\displaystyle \frac{2}{a} \sin \omega t$. The term $sin^{2}$ has a steady state and sinusoidal component:

<center>
$\sigma \approx 2 \displaystyle \left. \frac{d \Psi}{d \theta} \right|_{\theta = \hat{\theta}} \sin^{2} \left( \omega t \right) = \displaystyle \left. \frac{d \Psi}{d \theta} \right|_{\theta = \hat{\theta}} \left( 1 - \cos \left( 2 \omega t \right) \right)$
$\quad$
</center>

This low-pass filter removes the sinusoidal component from the demodulated signal, leaving an estimate of the gradient of the objective function with respect to the setpoint:

<center>
$\hat{\xi} \approx \displaystyle \left. \frac{d \Psi}{d \theta} \right|_{\theta = \hat{\theta}}$
$\quad$
</center>


In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt

from lib.ExtremumSeekingSimple1D import ExtremumSeekingSimple1D

from lib.System_Module import PassThroughSystem

from lib.Objective_Function_Module import ObjectiveFunction

from lib.Simulation_Module import Simulation

from lib.Plotting_Module import plot_es_results


### Example 1: Simulation of ES with simple passthrough system

In this example, a single 1D ESC minimizes the value of an objective function, by estimating the optimal input to an unknown system.

System: $y (t) = u (t)$

Objective function: $\left( y - y^{*} \right)^{2}$

Reference: $y^{*} = 2$

In [ ]:
# Setup and run simulation of simple passthrough system

def pass_through_system_01(u):
    return 1.0*u

def objective_function_01(y, ystar):
    return np.sum((y - ystar)**2)

# setup time
dT = 0.01 # simulation timestep
time = np.arange(0,30+dT,dT)

# desired system output (reference signal)
ystar = 2*np.ones(len(time))

PST01 = PassThroughSystem(time, dT, 1, 1, pass_through_system_01, ystar)

OBJ01 = ObjectiveFunction(time, dT, objective_function_01, ystar)

# create instance of ES Algorithm
ESC01 = ExtremumSeekingSimple1D(time, dT, 1, 0.2, 0.25, "minimize", name="1DES")

PST01.set_es_list([ESC01])
OBJ01.set_system_list([PST01])
# ESC01.set_objective_function([OBJ01])

sim = Simulation(time, dT, [PST01], [ESC01], [OBJ01])

sim.run_simulation()
    
print("Simulation Complete")
    
# plot_es_results(dT, time, [ESC01], OBJ01.y, OBJ01.ystar)
plot_es_results(time, dT, [sim], [PST01], [OBJ01], [ESC01], OBJ01.ystar)


### Example 2: Simulation of 1D ES with passthrough system

In this example, a single 1D ESC minimizes the value of an objective function, by estimating the optimal input to an unknown system.

System: $y(t) = 0.5u(t) + 4$

Objective function: $\left( y - y^{*} \right)^{2}$

Reference: $y^{*} = 2$

In [ ]:
# Setup and run simulation of simple passthrough system

def pass_through_system_01(u):
    return 0.5*u + 4

def objective_function_01(y, ystar):
    return np.sum((y - ystar)**2)

# setup time
dT = 0.01 # simulation timestep
time = np.arange(0,30+dT,dT)

# desired system output (reference signal)
ystar = 2*np.ones(len(time))

PST01 = PassThroughSystem(time, dT, 1, 1, pass_through_system_01, ystar)

OBJ01 = ObjectiveFunction(time, dT, objective_function_01, ystar)

# create instance of ES Algorithm
ESC01 = ExtremumSeekingSimple1D(time, dT, 1, 0.2, 0.5, "minimize", name="1DES")

PST01.set_es_list([ESC01])
OBJ01.set_system_list([PST01])
# ESC01.set_objective_function([OBJ01])

sim = Simulation(time, dT, [PST01], [ESC01], [OBJ01])

sim.run_simulation()
    
print("Simulation Complete")
    
# plot_es_results(dT, time, [ESC01], OBJ01.y, OBJ01.ystar)
plot_es_results(time, dT, [sim], [PST01], [OBJ01], [ESC01])


### Example 3: Simulation of ES with passthrough system with time varying reference

In this example, a single 1D ESC minimizes the value of an objective function with time varying reference.

System: $y(t) = 0.5u(t)$

Objective Function: $\left( y - y^{*} ( t ) \right)^{2}$

Reference:

$y^{*} (t) = 1$ for $0 \le t < 30$

$y^{*} (t) = 0$ for $30 \le t < 60$

$y^{*} (t) = (t - 60)/30$ for $60 \le t < 90$

$y^{*} (t) = \sin \left(\pi \left( t - 90 \right)\right)$ for $90 \le t \le 120$

In [ ]:
# Setup and run simulation of simple passthrough system

# Setup and run simulation of simple passthrough system

def pass_through_system_01(u):
    return 0.5*u

def objective_function_01(y, ystar):
    return np.sum((y - ystar)**2)

# setup time
dT = 0.01 # simulation timestep
time = np.arange(0,120+dT,dT)

# desired system output (reference signal)
# desired system output (reference signal)
ystar = np.zeros(len(time))
ystar[0:int(len(time)/4)] = 1
ystar[int(len(time)/4)] = 1
ystar[2*int(len(time)/4):3*int(len(time)/4)] = 1*(time[2*int(len(time)/4):3*int(len(time)/4)] - time[2*int(len(time)/4)])/time[1*int(len(time)/4)]
ystar[3*int(len(time)/4):4*int(len(time)/4)] = np.sin(0.5*2*np.pi/time[1*int(len(time)/4)]*(time[2*int(len(time)/4):3*int(len(time)/4)] - time[2*int(len(time)/4)]))

PST01 = PassThroughSystem(time, dT, 1, 1, pass_through_system_01, ystar)

OBJ01 = ObjectiveFunction(time, dT, objective_function_01, ystar)

# create instance of ES Algorithm
ESC01 = ExtremumSeekingSimple1D(time, dT, 1, 0.2, 0.5, "minimize", name="1DES")

PST01.set_es_list([ESC01])
OBJ01.set_system_list([PST01])
# ESC01.set_objective_function([OBJ01])

sim = Simulation(time, dT, [PST01], [ESC01], [OBJ01])

sim.run_simulation()
    
print("Simulation Complete")
    
# plot_es_results(dT, time, [ESC01], OBJ01.y, OBJ01.ystar)
plot_es_results(time, dT, [sim], [PST01], [OBJ01], [ESC01], OBJ01.ystar)


### Example 4: Simulation with 2 separate passthrough systems, 2 separate objective functions, and 2 separate ES controllers

In this example, a two 1D ESC respectively minimize the values of two separate objective functions, which are calculated using the respective outputs of two separate systems.

System(s): $y_{0} = 0.5 u_{0}$ and $y_{1} = 0.75 u_{1}$

Objective Function(s): $\Psi_{0} = \left( y_{0} - y_{0}^{*} \right)^{2}$ and $\Psi_{1} = \left( y_{1} - y_{1}^{*} \right)^{2}$

Reference(s): $y_{0}^{*} = 2$ and $y_{1}^{*} = -2$

The user should note that as the systems, objective functions, and ES controllers do not interact, this simulation can be rewritten as two separate simulations, each with corresponding system, objective function, and ES controller.


In [ ]:
# Setup and run simulation of simple passthrough system with multiple ES operating in parallel to minimize/maximize two separate objective functions

# Setup and run simulation of simple passthrough system

def pass_through_system_01(u):
    return 0.5*u

def pass_through_system_02(u):
    return 0.75*u

def objective_function_01(y, ystar):
    return np.sum((y - ystar)**2)

def objective_function_02(y, ystar):
    return np.sum((y - ystar)**2)

# setup time
dT = 0.01 # simulation timestep
time = np.arange(0,30+dT,dT)

# desired system output (reference signal)
ystar01 = 2*np.ones(len(time))
ystar02 = -2*np.ones(len(time))

PST01 = PassThroughSystem(time, dT, 1, 1, pass_through_system_01, ystar01)
PST02 = PassThroughSystem(time, dT, 1, 1, pass_through_system_02, ystar02)

OBJ01 = ObjectiveFunction(time, dT, objective_function_01, ystar01)
OBJ02 = ObjectiveFunction(time, dT, objective_function_02, ystar02)

# create instance of ES Algorithm
ESC01 = ExtremumSeekingSimple1D(time, dT, 1, 0.2, 0.5, "minimize", name="1DES_01")
ESC02 = ExtremumSeekingSimple1D(time, dT, 1.2, 0.2, 0.5, "minimize", name="1DES_02")

PST01.set_es_list([ESC01])
OBJ01.set_system_list([PST01])
# ESC01.set_objective_function([OBJ01])

PST02.set_es_list([ESC02])
OBJ02.set_system_list([PST02])
# ESC01.set_objective_function([OBJ01])

obj_to_es_map = np.array([0, 1])

sim = Simulation(time, dT, [PST01, PST02], [ESC01, ESC02], [OBJ01, OBJ02], obj_to_es_map)

sim.run_simulation()
    
print("Simulation Complete")
    
# plot_es_results(dT, time, [ESC01, ESC02], OBJ01.y, OBJ01.ystar)
plot_es_results(time, dT, [sim], [PST01, PST02], [OBJ01, OBJ02], [ESC01, ESC02], OBJ01.ystar)
    

### Example 5: Simulation with 2 passthrough systems, 1 objective function, and 2 separate ES controllers

In this example, a two 1D ESC minimize the value of one objective function, which is calculated from the outputs of two separate systems. The outputs of the 2 ES controllers form the input to the two systems, respectively.

Here subscripts denote the system, and ES controller.

System(s): $y_{0} = 0.5 u_{0}$ and $y_{1} = 0.75 u_{1}$

Objective Function: $\Psi = \left( y_{0} - y_{0}^{*} \right)^{2} + \left( y_{1} - y_{1}^{*} \right)^{2}$

Subscripts denote the system or ES controller index.


In [ ]:
# Setup and run simulation of simple passthrough system with multiple ES operating in parallel to minimize/maximize two separate objective functions

# Setup and run simulation of simple passthrough system

def pass_through_system_01(u):
    return 0.5*u

def pass_through_system_02(u):
    return 0.75*u

def objective_function(y, ystar):
    return np.sum((y - ystar)**2)


# setup time
dT = 0.01 # simulation timestep
time = np.arange(0,30+dT,dT)

# desired system output (reference signal)
ystar = 2*np.ones((2,len(time)))
ystar[0,:] = 1
ystar[1,:] = -1

PST01 = PassThroughSystem(time, dT, 1, 1, pass_through_system_01, ystar)
PST02 = PassThroughSystem(time, dT, 1, 1, pass_through_system_02, ystar)

OBJ01 = ObjectiveFunction(time, dT, objective_function, ystar)

# create instance of ES Algorithm
ESC01 = ExtremumSeekingSimple1D(time, dT, 1, 0.2, 0.5, "minimize", name="1DES_01")
ESC02 = ExtremumSeekingSimple1D(time, dT, 1.2, 0.2, 0.5, "minimize", name="1DES_02")

PST01.set_es_list([ESC01])
PST02.set_es_list([ESC02])

OBJ01.set_system_list([PST01, PST02])
obj_to_es_map = np.array([0, 0])

sim = Simulation(time, dT, [PST01, PST02], [ESC01, ESC02], [OBJ01], obj_to_es_map)

sim.run_simulation()
    
print("Simulation Complete")
    
# plot_es_results(dT, time, [ESC01, ESC02], OBJ01.y, OBJ01.ystar)
plot_es_results(time, dT, [sim], [PST01, PST02], [OBJ01], [ESC01, ESC02], OBJ01.ystar)
    

### Example 6: Simulation with 1 passthrough system, 1 objective functions, and 2 ES controllers that feed into the single system

In this example, two 1D ESC respectively minimize the value of one objective function, which is calculated using the outputs of one system. The outputs of the 2 ES controllers form the input to the system.

Here bold lowercase letters represent vectors, and subscripts on $u$ denote the ES controller.

Unknown system: $\textbf{y} = \begin{bmatrix} u_{0} \\ u_{1} \\ u_{0} + 1.5 u_{1} \end{bmatrix}$

Objective Function: $\Psi = \left( \textbf{y} - \textbf{y}^{*} \right)^{T} \begin{bmatrix} 0.01 & 0 & 0 \\ 0 & 0.01 & 0 \\ 0 & 0 & 1 \end{bmatrix} \left( \textbf{y} - \textbf{y}^{*} \right)$

Reference: $\textbf{y}^{*} = \left[ 0, 0, 4 \right]^{T}$


In [ ]:
# Setup and run simulation of simple passthrough system with multiple ES operating in parallel to minimize/maximize two separate objective functions

# Setup and run simulation of simple passthrough system

def pass_through_system_01(u):
    return np.array([[1.0*u[0], 1.0*u[1], (1.0*u[0] - 1.5*u[1])]]).T

def objective_function(y, ystar):
    return np.sum(np.array([1e-2, 1e-2, 1e0])*(y - ystar)**2)


# setup time
dT = 0.01 # simulation timestep
time = np.arange(0,60+dT,dT)

# desired system output (reference signal)
ystar = np.zeros((3,len(time)))
# ystar[0,:] = 0
# ystar[1,:] = 0
ystar[2,:] = 4

PST01 = PassThroughSystem(time, dT, 2, 3, pass_through_system_01, ystar)

OBJ01 = ObjectiveFunction(time, dT, objective_function, ystar)

# create instance of ES Algorithm
ESC01 = ExtremumSeekingSimple1D(time, dT, 1, 0.2, 0.2, "minimize", name="1DES_01")
ESC02 = ExtremumSeekingSimple1D(time, dT, 1.2, 0.2, 0.2, "minimize", name="1DES_02")

PST01.set_es_list([ESC01, ESC02])
# PST02.set_es_list([ESC02])

OBJ01.set_system_list([PST01])
obj_to_es_map = np.array([0, 0])

sim = Simulation(time, dT, [PST01], [ESC01, ESC02], [OBJ01], obj_to_es_map)

sim.run_simulation()
    
print("Simulation Complete")
    
plot_es_results(time, dT, [sim], [PST01], [OBJ01], [ESC01, ESC02], OBJ01.ystar)

    